### Prepare Data

In [1]:
import string
from random import shuffle
import numpy as np
x = [[i] for i in range(10)]


FEMALE_FILE = 'Baby_names/female.txt';
MALE_FILE = 'Baby_names/male.txt';
N_ALPHABET = 28; #Add 2 extra for encoding the end of name EON and the dash '-'
MAX_CHARS = 16; #Assume no name is longer than 16 chars

alphabet_id = {}
for i in xrange(26):
    alphabet_id[string.lowercase[i]] = i;

alphabet_id['-'] = 26;
alphabet_id['\n'] = 27;

id2char = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',\
           'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', \
           's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '-', '\n'];

def str2onehot(string):
    string = string.lower();
    result = np.zeros((MAX_CHARS, N_ALPHABET));
    
    chars = list(string);
    idx = [alphabet_id[char] for char in chars];
    result[range(len(string)), idx] = 1;
    return result;
input_male = []
target_male = []
sequence_len = []
with open(MALE_FILE) as fp:
    for line in fp:
        if (line[0]!="#" and line[0]!="\n"):
            input_male.append(str2onehot(line[:-1]))
            target_male.append(str2onehot(line[1:]))
            sequence_len.append(len(line)-1);
n = len(input_male);
idx = [i for i in range(n)]
shuffle(idx)
input_male = [input_male[i] for i in idx]
target_male = [target_male[i] for i in idx]
sequence_len = [sequence_len[i] for i in idx]

### Build the LSTM graph

In [2]:
import tensorflow as tf
tf.reset_default_graph()

max_chars = max([name.shape[0] for name in input_male]);
#Placeholder for input and target, [Batch Size, Sequence Length, Input Dimension]
data = tf.placeholder(tf.float32, [None, max_chars, N_ALPHABET])
target = tf.placeholder(tf.float32, [None, max_chars, N_ALPHABET])

num_hidden = 150
time_steps = max_chars

lstmCell = tf.nn.rnn_cell.BasicLSTMCell(num_hidden, forget_bias=1.0, state_is_tuple=True)
#prediction one char
x = tf.placeholder(tf.float32, [1, N_ALPHABET])
m = lstmCell.zero_state(1, dtype=tf.float32)
out_char, state = lstmCell(x, m);


seqlen = tf.placeholder(tf.int32, [None])

#unroll the LSTM network
outputs, states = tf.nn.dynamic_rnn(cell=lstmCell, inputs=data, sequence_length=seqlen, dtype=tf.float32)

# Define weights
out_size = N_ALPHABET;

# Weights for each timestep output, using the same weight
weight = tf.Variable(tf.truncated_normal([num_hidden, out_size], stddev=0.1))
bias = tf.Variable(tf.constant(0.1, shape=[out_size]))
char_prediction = tf.nn.softmax(tf.matmul(out_char, weight) + bias)

outputs = tf.reshape(outputs, [-1, num_hidden])
prediction = tf.nn.softmax(tf.matmul(outputs, weight) + bias)
prediction = tf.reshape(prediction, [-1, time_steps, out_size])

cross_entropy = -tf.reduce_sum(target * tf.log(prediction))
#cross_entropy = tf.reduce_mean(cross_entropy)

#cross_entropy = -tf.reduce_sum(target * tf.log(prediction))
final_prediction = tf.argmax(prediction, 2)
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

mistakes = tf.not_equal(tf.argmax(target, 2), tf.argmax(prediction, 2))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

### Execute the graph

In [9]:
from numpy import random
def pred2name(pred):
    #ndarray of n_time_steps * N_ALPHABET
    name = [id2char[char_id] for char_id in pred];
    return "".join(name)

init_op = tf.initialize_all_variables()
#sess = tf.Session()
#sess.run(init_op)

batch_size = 100
no_of_batches = int(len(input_male)/batch_size)
epoch = 1000

for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out, seql = input_male[ptr:ptr+batch_size], target_male[ptr:ptr+batch_size], sequence_len[ptr:ptr+batch_size]   
        ptr+=batch_size
        result = sess.run([minimize, cross_entropy, final_prediction, error], {data: inp, target: out, seqlen: seql})
        #print(result[3])
        
    print(result[1])
    prediction = result[2];
    idx = random.choice(range(prediction.shape[0]), size=2);
    preds = prediction[idx]
    for pred in preds:
            sample_name = pred2name(pred)
            print repr(sample_name)
            
    print "Epoch - ",str(i)

#sess.close();



519.076
'arbhhel\nuuuuuuuu'
'aodk\n\nuuuuuuuuuu'
Epoch -  0
518.978
'ehnnthon\nuuuuuuu'
'amtrr\nuuuuuuuuuu'
Epoch -  1
518.993
'aelwond\nuuuuuuuu'
'aaine\nuuuuuuuuuu'
Epoch -  2
518.858
'eck\nas\nuuuuuuuuu'
'irnel\nuuuuuuuuuu'
Epoch -  3
518.913
'eony\nuuuuuuuuuuu'
'leetros\nuuuuuuuu'
Epoch -  4
518.739
'ldonins\nuuuuuuuu'
'hr\nuuuuuuuuuuuuu'
Epoch -  5
518.834
'ann\ne\nuuuuuuuuuu'
'aoaneuuuuuuuuuuu'
Epoch -  6
518.623
'odao\nuuuuuuuuuuu'
'anndict\nuuuuuuuu'
Epoch -  7
518.757
'artis\nuuuuuuuuuu'
'ali\nuuuuuuuuuuuu'
Epoch -  8
518.511
'ecuuuuuuuuuuuuuu'
'irnel\nuuuuuuuuuu'
Epoch -  9
518.678
'erein\nuuuuuuuuuu'
'hrde\nuuuuuuuuuuu'
Epoch -  10
518.402
'eoty\nuuuuuuuuuuu'
'odelph\nuuuuuuuuu'
Epoch -  11
518.598
'ethaniel\nuuuuuuu'
'eh\nuuuuuuuuuuuuu'
Epoch -  12
518.297
'odao\nuuuuuuuuuuu'
'ethaniel\nuuuuuuu'
Epoch -  13
518.515
'aone\nuuuuuuuuuuu'
'eoty\nuuuuuuuuuuu'
Epoch -  14
518.197
'ehnn\nuuuuuuuuuuu'
'ehnnthon\nuuuuuuu'
Epoch -  15
518.428
'hannceyuuuuuuuuu'
'arrnner\nuuuuuuuu'
Ep

## Generate Baby Name

In [11]:
batch_size = 1;
input_size = N_ALPHABET;

n = 50;
def char2onehot(char):
    char = char.lower();
    result = np.zeros((1, N_ALPHABET));
    idx = alphabet_id[char];
    result[0, idx] = 1;
    return result;

def samplePred(pred):
    dist = pred[0, ]
    #idx_min = dist.argsort()[:(len(dist)/2)]
    #pred[0, idx_min] = 0
    #pred[0, ] = pred[0, ]/sum(pred[0, ])
    char_id = np.random.choice(N_ALPHABET, 1, p=pred[0, ])
    return char_id
    
for i in range (n):
    last_char = "a";
    count = 0
    name = last_char;
    last_input = char2onehot(last_char);
    current_state = (np.zeros((1, num_hidden)), np.zeros((1, num_hidden)));
    while (last_char != '\n') and (count < 17):
        count = count+1;
        res = sess.run([char_prediction, state], {x:last_input, m:current_state} );
        char_id = samplePred(res[0])
        last_char = id2char[char_id];
        name += last_char;
        current_state = res[1];
    print name;


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:30: DeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


aizobjypbgffpq

as

alehbwijsxpdpxahvk
axj

asuhxwfulcublvtskx
amuvupusb-xr

a-cthbfslnnmfihdks
ayprvutiqhbtlcgmjw
aryfg

apyxjrpjf-nfrmtfqc
autegxsrlrlcvfpxev
av-prrokgd

awuexvlvfajjcfo-gb
ayyjifkfxtvactpztk
adi-wgelxfv-

auddpmlf

aofrbfrmpumttrkxpb
avjjxcxnfhfgpnftfl
aubdddbfjvlvkhcrdu
aj

ayhvptpzzpjlxftfht
aawb-hmblpttl-ttvb
asvfwfa-fgtftcsfvf
aztroythppdcrsdb-j
ansvlxtursbzb

aucpjoschdffpllgxp
aaqbv

ayqlfbfknyfbvffcbx
aiiollavbkcftcplvc
aojrnsdufdfgbbvqmn
aymnrdwfvvtlmclndf
afjfib

avstxvxllpqfhstlgm
aldleedfp

afpthvhtjpfzsdfltd
apz

akjhtngrlfbebelqfp
aknehepprtlbbghvtx
anpbvallcpu

awpdfb

aouufklgoq

auilvlaghrdh-oiglg
ajjirg

apdhvfaitljhfnnvzb
arfqr

a

ahhu

aks

avcnpuwgdrcl-vlkfc
arrndqvfxfq



In [68]:
sess.close();

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.]])

In [44]:
from random import shuffle
idx = [i for i in range(10)]
shuffle(idx)


In [4]:
%autosave 10000

Autosaving every 10000 seconds


In [43]:
type(x)

NoneType

In [15]:
sess.close()

In [99]:
a = np.array([9, 4, 4, 3, 3, 9, 0, 4, 6, 0])
ind = np.argpartition(a, -5)[-5:]
ind
a[ind]

array([4, 4, 9, 6, 9])